In [15]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim

In [16]:
def normalize_features(X_train, X_test):
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train_norm = scaler.transform(X_train)
    X_test_norm = scaler.transform(X_test)
    return X_train_norm, X_test_norm

def PCC(x, y):
    A = x - np.mean(x)
    B = y - np.mean(y)
    return np.dot(A, B) / np.sqrt(np.dot(A, A) * np.dot(B, B))

def RMSE(x, y):
    m = x.shape[0]
    return np.sqrt(np.dot(x - y, x - y) / m)

In [17]:
class DecisionTreeRegressor():
    def __init__(self, max_depth = 5, current_depth = 1, max_features = None):
        self.max_depth = max_depth
        self.current_depth = current_depth
        self.left_tree = None
        self.right_tree = None
        self.max_features = max_features

    def fit(self, X, y):
        self.X = X
        self.y = y
        self.n_features = X.shape[1]
        self.n_samples = X.shape[0]
        if self.current_depth <= self.max_depth:
            self.rss = self.rss_calculation(self.y)
            self.best_feature_id, self.best_gain, self.best_split_value = self.find_best_split()
            if self.best_gain > 0:
                self.split_trees()

    def split_trees(self):
        self.left_tree = DecisionTreeRegressor(max_depth = self.max_depth, current_depth = self.current_depth + 1)
        self.right_tree = DecisionTreeRegressor(max_depth = self.max_depth, current_depth = self.current_depth + 1)
        best_feature_values = self.X[:, self.best_feature_id]
        left_indices = np.where(best_feature_values < self.best_split_value)
        right_indices = np.where(best_feature_values >= self.best_split_value)
        left_tree_X, left_tree_y = self.X[left_indices], self.y[left_indices]
        right_tree_X, right_tree_y = self.X[right_indices], self.y[right_indices]
        self.left_tree.fit(left_tree_X, left_tree_y)
        self.right_tree.fit(right_tree_X, right_tree_y)

    def find_best_split(self):
        best_feature_id = None
        best_gain = 0
        best_split_value = None
        if self.max_features is None:
            for feature_id in range(self.n_features):
                current_gain, current_split_value = self.find_best_split_one_feature(feature_id)
                if current_gain is None:
                    continue
                if best_gain < current_gain:
                    best_feature_id = feature_id
                    best_gain = current_gain
                    best_split_value = current_split_value
        elif self.max_features == 'sqrt':
            rng = np.random.default_rng()
            sampled_features = rng.choice(self.X.shape[1], int(np.sqrt(self.X.shape[1])), replace = False)
            for feature_id in sampled_features:
                current_gain, current_split_value = self.find_best_split_one_feature(feature_id)
                if current_gain is None:
                    continue
                if best_gain < current_gain:
                    best_feature_id = feature_id
                    best_gain = current_gain
                    best_split_value = current_split_value
        return best_feature_id, best_gain, best_split_value

    def find_best_split_one_feature(self, feature_id):
        feature_values = self.X[:, feature_id]
        unique_feature_values = np.unique(feature_values)
        best_gain = 0.0
        best_split_value = None
        if len(unique_feature_values) == 1:
            return best_gain, best_split_value
        for fea_val in unique_feature_values:
            left_indices, right_indices = np.where(feature_values < fea_val), np.where(feature_values >= fea_val)
            left_tree_y, right_tree_y = self.y[left_indices], self.y[right_indices]
            left_rss, right_rss = self.rss_calculation(left_tree_y), self.rss_calculation(right_tree_y)
            current_gain = self.rss - left_rss - right_rss
            if best_gain < current_gain:
                best_gain = current_gain
                best_split_value = fea_val
        return best_gain, best_split_value

    def rss_calculation(self, y):
        if y.size == 0 or y is None:
            return 0
        return np.sum((y - np.mean(y))**2)

    def predict(self, X_test):
        n_test = X_test.shape[0]
        ypred = np.zeros(n_test, dtype = int)
        for i in range(n_test):
            ypred[i] = self.tree_propogation(X_test[i])
        return ypred

    def tree_propogation(self, feature):
        if self.is_leaf_node():
            return self.predict_label()
        if feature[self.best_feature_id] < self.best_split_value:
            child_tree = self.left_tree
        else:
            child_tree = self.right_tree
        return child_tree.tree_propogation(feature)

    def predict_label(self):
        return np.mean(self.y)

    def is_leaf_node(self):
        return self.left_tree is None 

In [18]:
class random_forest_regressor:
    def __init__(self, n_estimators = 10):
        self.n_estimators = n_estimators
        self.forest = []
        for i in range(self.n_estimators):
            self.forest.append(DecisionTreeRegressor(max_depth = 20, max_features='sqrt'))

    def fit(self, X, y):
        '''bagging'''
        self.X = X
        self.y = y
        self.N = X.shape[1]
        self.M = X.shape[0]

        self.trees_idx = np.random.randint(0, self.M, size = (self.n_estimators, self.M))
        for i, itree in enumerate(self.forest):
            itree.fit(self.X[self.trees_idx[i]], self.y[self.trees_idx[i]])
            print("The {}th tree is built".format(i))

    def predict(self, Xtest):
        n_test = Xtest.shape[0]
        ypred = np.zeros((self.n_estimators, n_test))
        for i, itree in enumerate(self.forest):
            ypred[i, :] = itree.predict(Xtest)
        return np.mean(ypred, axis = 0) # stats.mode get the most common element along a certain axis

In [19]:
class GradientBoostingRegressor():
    def __init__(self, n_estimators = 10, lr = 1.):
        self.n_estimators = n_estimators
        self.lr = lr
        self.regressors = []
        for i in range(self.n_estimators):
            self.regressors.append(DecisionTreeRegressor())

    def fit(self, X, y):
        y_pred = np.full(y.size, np.mean(y))
        self.y_pred0 = np.mean(y)
        residue = y - y_pred

        for i in np.arange(self.n_estimators):
            self.regressors[i].fit(X, residue)
            ipred = self.regressors[i].predict(X)
            y_pred = y_pred + self.lr * ipred
            residue = y - y_pred

    def predict(self, Xtest):
        y_pred = np.full(Xtest.shape[0], self.y_pred0)
        for i in np.arange(self.n_estimators):
            y_pred += self.lr * (self.regressors[i].predict(Xtest))
        return y_pred

In [20]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(108, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.layers(x)

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    current_loss = 0.0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        data, target = data.float(), target.float()
        target = target.reshape((target.shape[0], 1))

        optimizer.zero_grad()
        output = model(data)
        loss = loss_function(output, target)
        loss.backward()
        optimizer.step()
        current_loss += loss.item()

        if epoch % 5 == 0:
            print('Train Epoch: {} [{}/{}]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset), loss.item()))
            current_loss = 0.0

def test(model, device, epoch, test_loader):
    model.eval()
    test_loss = 0
    yvals = test_loader.dataset.yvals().detach().numpy()
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)
            data, target = data.float(), target.float()
            target = target.reshape((target.shape[0], 1))
            
            output = model(data)
            test_loss += loss_function(output, target).item()

            if batch_idx == 0:
                preds = output.detach().numpy()
            else:
                preds = np.append(preds, output.detach().numpy())
            
    test_loss /= len(test_loader.dataset)
    
    if epoch %5 == 0:
        print('\n Test set: Average loss: {:.4f}'.format(test_loss))
        print('RMSE is {} and PCC is {}'.format(RMSE(preds, yvals), PCC(preds, yvals)))
        

In [21]:
df_train = pd.read_csv('Binding_energy_train.csv')
f_train = df_train.values
df_test = pd.read_csv('Binding_energy_test.csv')
f_test = df_test.values

Xtrain = f_train[:,1:]
ytrain = f_train[:,0]
Xtest = f_test[:,1:]
ytest = f_test[:,0]

Xtrain_norm, Xtest_norm = normalize_features(Xtrain, Xtest)

In [24]:
RF = random_forest_regressor(n_estimators = 10)
RF.fit(Xtrain_norm, ytrain)
ypred = RF.predict(Xtest_norm)
rmse = RMSE(ypred, ytest)
pcc = PCC(ypred, ytest)

print("For Random Forest, the RMSE is {:.2f} and the PCC is {:.2f}".format(rmse, pcc))

The 0th tree is built
The 1th tree is built
The 2th tree is built
The 3th tree is built
The 4th tree is built
The 5th tree is built
The 6th tree is built
The 7th tree is built
The 8th tree is built
The 9th tree is built
For Random Forest, the RMSE is 1.55 and the PCC is 0.76


In [22]:
GBTR = GradientBoostingRegressor(n_estimators = 10)
GBTR.fit(Xtrain_norm, ytrain)
ypred = GBTR.predict(Xtest_norm)

rmse = RMSE(ypred, ytest)
pcc = PCC(ypred, ytest)

print("For Gradient Boosting, the RMSE is {:.2f} and the PCC is {:.2f}".format(rmse, pcc))

y sizes are (290,) and (290,)
For Gradient Boosting, the RMSE is 1.66 and the PCC is 0.66


In [14]:
class InitializeDataset(torch.utils.data.Dataset):
    def __init__(self, X, y, scale_data = True):
        if not torch.is_tensor(X) and not torch.is_tensor(y):
            if scale_data:
                X = StandardScaler().fit_transform(X)
            self.X = torch.from_numpy(X)
            self.y = torch.from_numpy(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        return self.X[i], self.y[i]

    def yvals(self):
        return self.y

device = torch.device("cpu")
train_data = InitializeDataset(Xtrain, ytrain)
test_data = InitializeDataset(Xtest, ytest)
train_loader = torch.utils.data.DataLoader(dataset = train_data, batch_size = 500, shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_data, batch_size = 16, shuffle = False)

model = MLP().to(device)
loss_function = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr = 0.01, amsgrad = False)
for epoch in range(1, 15 + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, epoch, test_loader)


Train Epoch: 5 [0/3767]	Loss: 1.481058
Train Epoch: 5 [500/3767]	Loss: 1.500421
Train Epoch: 5 [1000/3767]	Loss: 1.430969
Train Epoch: 5 [1500/3767]	Loss: 1.587077
Train Epoch: 5 [2000/3767]	Loss: 1.531697
Train Epoch: 5 [2500/3767]	Loss: 1.544250
Train Epoch: 5 [3000/3767]	Loss: 1.391415
Train Epoch: 5 [1869/3767]	Loss: 1.514726

 Test set: Average loss: 0.1141
RMSE is 2.1425028002038635 and PCC is 0.4126030743217336
Train Epoch: 10 [0/3767]	Loss: 1.243402
Train Epoch: 10 [500/3767]	Loss: 1.294141
Train Epoch: 10 [1000/3767]	Loss: 1.344394
Train Epoch: 10 [1500/3767]	Loss: 1.286102
Train Epoch: 10 [2000/3767]	Loss: 1.295546
Train Epoch: 10 [2500/3767]	Loss: 1.285979
Train Epoch: 10 [3000/3767]	Loss: 1.356080
Train Epoch: 10 [1869/3767]	Loss: 1.276915

 Test set: Average loss: 0.1023
RMSE is 1.948965366797737 and PCC is 0.48501413072452654
Train Epoch: 15 [0/3767]	Loss: 1.238486
Train Epoch: 15 [500/3767]	Loss: 1.215631
Train Epoch: 15 [1000/3767]	Loss: 1.233347
Train Epoch: 15 [1500/3